In [ ]:
#This code processes multiple picasso files to filter coordinates based on their parameters
#File needed is the hdf5 file generated from picasso after the localization step

In [2]:
import pandas as pd
import numpy as np
import h5py
import seaborn as sns

file_loc=list([f'./image{i}.hdf5' for i in range(1,2)])   
result_loc=list([f'image_{i}' for i in range(1,2)])
file_loc 

def loop_function(file_name,result_name,threshold_sx_lower,threshold_sx_upper,threshold_sy_lower,threshold_sy_upper,upper_lpx,upper_lpy):

    #Reading hdf5 files from picasso
    df_initial=pd.DataFrame(np.array(h5py.File(file_name)['locs']))
    df_2=df_initial[["frame","x","y","photons","sx","sy","lpx","lpy","group"]]

    #Filtering sx and sy
    df_filtered_sx=df_2[(df_2 ['sx'] > threshold_sx_lower) & (df_2 ['sx'] < threshold_sx_upper)]
    df_filtered_sx_sy=df_filtered_sx[(df_filtered_sx ['sy'] > threshold_sy_lower) & (df_filtered_sx ['sy'] < threshold_sy_upper)]
    fig_3=sns.displot(data=df_filtered_sx, x="sx", kde=True)
    fig_4=sns.displot(data=df_filtered_sx_sy, x="sy", kde=True)
    fig_3.savefig(f'./{result_name}_hist_sx.jpg')
    fig_4.savefig(f'./{result_name}_hist_sy.jpg')

    #Filtering lpx and lpy
    df_filtered_sx_sy_lpx=df_filtered_sx_sy[(df_filtered_sx_sy ['lpx'] < upper_lpx)]
    df_filtered_sx_sy_lpx_lpy=df_filtered_sx_sy_lpx[(df_filtered_sx_sy_lpx ['lpy'] < upper_lpy)]
    fig_5=sns.displot(data=df_filtered_sx_sy_lpx, x="lpx", kde=True)
    fig_6=sns.displot(data=df_filtered_sx_sy_lpx_lpy, x="lpy", kde=True)
    fig_5.savefig(f'./{result_name}_hist_lpx.jpg')
    fig_6.savefig(f'./{result_name}_hist_lpy.jpg')

    #You can also add here other parameters of your choosing to filter (such as photons)

    #Save output
    df_filtered_sx_sy_lpx_lpy.to_csv(f'{result_name}_filtered.csv',index=False)
    return 0

#Loop to process multiple files
#Values needed are as follows: threshold_sx_lower,threshold_sx_upper,threshold_sy_lower,threshold_sy_upper,upper_lpx,upper_lpy
for i,j in zip(file_loc,result_loc):
    loop_function(i,j,1.2,2.2,1.2,2.2,0.3,0.3)


In [ ]:
import pandas as pd
import numpy as np
import h5py
import seaborn as sns
import matplotlib.pyplot as plt

def loop_function(file_name, result_name, thresholds):
    """
    Process a single file in h5py format and save filtered data and histograms.

    Parameters:
    - file_name: str, path to the HDF5 file
    - result_name: str, name for the result files
    - thresholds: dict, dictionary containing threshold values for filtering
    """
    # Reading hdf5 files from picasso
    df_initial = pd.DataFrame(np.array(h5py.File(file_name)['locs']))
    df_2 = df_initial[["frame", "x", "y", "photons", "sx", "sy", "lpx", "lpy", "group"]]

    # Filtering based on thresholds
    filters = (
        (df_2['sx'].between(*thresholds['sx'])) &
        (df_2['sy'].between(*thresholds['sy'])) &
        (df_2['lpx'] < thresholds['upper_lpx']) &
        (df_2['lpy'] < thresholds['upper_lpy'])
    )
    df_filtered = df_2[filters]

    # Plotting and saving histograms
    for col in ['sx', 'sy', 'lpx', 'lpy']:
        fig = sns.displot(data=df_filtered, x=col, kde=True)
        fig.savefig(f'./{result_name}_hist_{col}.jpg')
        plt.close(fig)

    # Save filtered data to CSV
    df_filtered.to_csv(f'{result_name}_filtered.csv', index=False)

# Loop to process multiple files
# Values needed are as follows: threshold_sx_lower, threshold_sx_upper, threshold_sy_lower, threshold_sy_upper, upper_lpx, upper_lpy
file_loc = [f'./image{i}.hdf5' for i in range(1, 2)]
result_loc = [f'image_{i}' for i in range(1, 2)]
threshold_values = {
    'sx': (1.2, 2.2),
    'sy': (1.2, 2.2),
    'upper_lpx': 0.3,
    'upper_lpy': 0.3
}

for file_path, result_name in zip(file_loc, result_loc):
    loop_function(file_path, result_name, threshold_values)